In [125]:
import numpy as np
import pandas as pd
from pathlib import Path
import glob

import matplotlib.pyplot as plt
%matplotlib notebook

from NeuNorm.normalization import Normalization

# Instructions 

Here we define the only 2 input parameters to the program
 * input_folder
 * output_folder

In [4]:
input_folder = "data/raw_data/"
assert Path(input_folder).exists()

In [5]:
output_folder = 'data/corrected_data_calculated'
assert Path(output_folder).exists()

# Setup 

In [36]:
shutter_dict = {}

### ShutterCount.txt 

First, the program will locate, in that folder, the file names that ends by *_ShutterCount.txt* and read it in order to isolate the number of count for each shutter.

__Handling errors__: If file can not be located, error should be raised ("Missing ShutterCount.txt in folder")

In [37]:
shutter_count_file = glob.glob(input_folder + '/*_ShutterCount.txt')[0]
assert Path(shutter_count_file).exists()

In [38]:
shutter_count_pd = pd.read_csv(shutter_count_file, sep='\t', names=['shutter_index', 
                                                                    'shutter_counts'])

# isolate only the shutter counts
shutter_counts = shutter_count_pd['shutter_counts']
for shutter_index, shutter_counts in enumerate(shutter_counts):
    if shutter_counts == 0:
        break
    shutter_dict[shutter_index] = {'counts': shutter_counts}

### ShutterTimes.txt

This file needs to be retrieved and read as well in order to figure out when the various frames start and end (in microS)

In [32]:
shutter_time_file = glob.glob(input_folder + '/*_ShutterTimes.txt')[0]
assert Path(shutter_time_file).exists()

In [112]:
shutter_time_pd = pd.read_csv(shutter_time_file, sep='\t', names=['shutter_index',
                                                                 'start_frame',
                                                                 'end_frame'])
# isolate only the start and end
shutter_start_time = shutter_time_pd['start_frame']
shutter_end_time = shutter_time_pd['end_frame']

global_offset = 0
for shutter_index, start_time in enumerate(shutter_start_time):
    if shutter_end_time[shutter_index] == 0:
        break
    shutter_dict[shutter_index]['start_time'] = start_time + global_offset
    global_offset += start_time
    shutter_dict[shutter_index]['end_time'] = shutter_end_time[shutter_index] + global_offset
    global_offset += shutter_end_time[shutter_index]

### Spectra.txt

In [113]:
spectra_file = glob.glob(input_folder + '/*_Spectra.txt')[0]
assert Path(spectra_file).exists()

In [114]:
spectra_file_pd = pd.read_csv(spectra_file, sep='\t', names=['shutter_time',
                                                             'counts'])

#we are really only interested by the first column
spectra_time_array = np.array(spectra_file_pd['shutter_time'])

### When does each frame starts and ends?

We will use the data retrieve in the specta.txt and ShutterTimes.txt to figure out which file belongs to which frame

In [115]:
def find_nearest_index(array, value):
    perfect_match = (0 in (np.abs(np.array(array) - value)))
    idx = (np.abs(np.array(array) - value)).argmin()
    return perfect_match, idx

In [116]:
for _frame_index in shutter_dict.keys():
    start_time = shutter_dict[_frame_index]['start_time']
    perfect_match, nearest_start_index = find_nearest_index(spectra_time_array, start_time)
    # for start index, no need to check for perfect match as we always get the next index (if not perfect match)
    shutter_dict[_frame_index]['start_index'] = nearest_start_index

    end_time = shutter_dict[_frame_index]['end_time']
    perfect_match, nearest_end_index = find_nearest_index(spectra_time_array, end_time)
    # we need to check for perfect match or not. if not, we need to use the previous index
    nearest_index = nearest_end_index if perfect_match else nearest_end_index - 1
    shutter_dict[_frame_index]['end_index'] = nearest_index

# Recap 

In [131]:
import pprint
pprint.pprint(shutter_dict)

{0: {'counts': 433151,
     'end_index': 242,
     'end_time': 0.00249952,
     'start_index': 0,
     'start_time': 9.6e-07},
 1: {'counts': 445394,
     'end_index': 525,
     'end_time': 0.00579792,
     'start_index': 244,
     'start_time': 0.0029},
 2: {'counts': 446435,
     'end_index': 1472,
     'end_time': 0.01589728,
     'start_index': 527,
     'start_time': 0.0062}}


## Load images 

In [120]:
list_input_files = glob.glob(input_folder + '/*.fits')

# removing _SummedImg.fits
clean_list_input_files = []
for _file in list_input_files:
    if "_SummedImg.fits" in _file:
        continue
    clean_list_input_files.append(_file)

In [127]:
o_norm = Normalization()
o_norm.load(file=list_input_files, notebook=True)

# Code 

In [128]:
raw_data = o_norm.data['sample']['data']
pixel_occupancy_probablity = []

In [ ]:
for shutter_index in shutter_dict.keys():
    